In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

In [65]:
data_path= os.path.join(os.getcwd(), "data")
model_path= os.path.join(os.getcwd(), "models")
embeddings_path = os.path.join(os.getcwd(), "embeddings")

In [66]:
df = pd.read_parquet(os.path.join(data_path, "clean_df.pq"))

In [67]:
item_desc_emb_df = pd.read_parquet(os.path.join(embeddings_path, "item_desc_emb_df.pq"))
item_price_df = pd.read_parquet(os.path.join(embeddings_path, "item_price_df.pq"))
user_review_emb_df = pd.read_parquet(os.path.join(embeddings_path, "user_review_emb_df.pq"))
user_item_emb_df = pd.read_parquet(os.path.join(embeddings_path, "user_item_emb_df.pq"))

In [105]:
model = load_model(os.path.join(model_path, "best_model.keras"))

In [69]:
user_names = user_review_emb_df.index
size = len(item_desc_emb_df)

In [79]:
def get_recommendations(user):
    if user in user_names:
        user_review_emb = np.tile(user_review_emb_df.loc[user], (size, 1))
        user_item_emb = np.tile(user_item_emb_df.loc[user], (size, 1))
        user_set = (user_review_emb, user_item_emb, item_desc_emb_df, item_price_df)
        predictions = model.predict(user_set)
        top_indices = np.argsort(-predictions[:, 0])[:5]  
        recommendations = item_desc_emb_df.index[top_indices].tolist()    
    else:
        cold_start = df[df["rating"] == 5].groupby("itemName")["reviewTime"].agg(["size", "max"])
        recommendations = cold_start[
            cold_start["max"] > cold_start["max"].max() - pd.Timedelta(days=30)
        ].sort_values("size", ascending=False)[:5].index.to_list()
    return recommendations

In [95]:
user = user_names[33]
user

'1 online shopper'

In [109]:
get_recommendations("oleg")

3405/3405 ━━━━━━━━━━━━━━━━━━━━ 2s 546us/step


['hhip 39000102 r8 end mill holder 38quot',
 'ranger sticky embossing powder 21 grams epj35275',
 '10x13 self seal catalog envelopescolor bright white large envelope10 x 13 open endpck 35 white',
 '1 inch round thank you labels with black hearts 1000 stickers per roll 2 rolls',
 'imbaprice 000 4 x 8 kraft bubble mailers padded envelopes total 50 envelope']